In [ ]:
from __future__ import print_function
from __future__ import unicode_literals
from collections import OrderedDict
import copy
import io
import json
import logging; logging.basicConfig(level=logging.DEBUG)
logging.getLogger('git.cmd').setLevel(logging.WARNING)
import os
import os.path
import pkg_resources
import re
import sys
import tempfile as tf

from ruamel.yaml import YAML
from ruamel.yaml.constructor import DuplicateKeyError
import conda_helpers as ch
import git
import path_helpers as ph
import pydash
import pydash as _py
import semantic_version
import yaml

crum_src = ph.path('.').joinpath('_tools', 'crum', 'src').realpath()
if crum_src not in sys.path:
    sys.path.insert(0, crum_src)
from crum.render import iter_deep
from crum.render_recipes import main, render_recipes
from crum.bootstrap import recipe_objs
from crum.git_tools import roll_back_bump, bump_requirements, commit_recipes

In [ ]:
recipes = [ph.path(line_i.strip()) for line_i in ph.path('bootstrap-build-order.txt').lines()]
args = recipes + '-- -c file:///C:\dbot -c conda-forge'.split(' ')
results = render_recipes('.cache', recipes, '-c file:///C:\dbot -c conda-forge'.split(' '))
recipe_objs_ = OrderedDict([(k, recipe_objs(v)) for k, v in results.items()])

In [ ]:
# roll_back_bump(repo)

In [ ]:
start_subpackage_info, subpackage_info = bump_requirements(recipe_objs_)
repo = git.Repo('.')
commit_recipes(repo, dry_run=False)

In [ ]:
from _tools.gitpython_helpers import traverse

In [ ]:
repo_root = ph.path(repo.working_tree_dir)

recipes_to_build = []
for repo_i in traverse(repo):
    try:
        repo_path_i = repo_root.relpathto(repo_i.working_tree_dir)
        print(repo_path_i, repo_i.git.describe('--tags', '--dirty'))
    except:
        pass
#     if repo_i.git.diff('origin/master'):
#         repo_path_i = repo_root.relpathto(repo_i.working_tree_dir)
#         print(repo_path_i, 'needs to be pushed')
#         recipes_to_build.append(repo_path_i)
#     try:
#         repo_i.git.push()
#     except git.GitCommandError as e:
#         print(repo_i, repo_root.relpathto(repo_i.working_tree_dir), e)

print(' '.join(r for r in recipes if ph.path(r).normpath() in recipes_to_build))